In [1]:
import os
import os.path

from urllib.parse import urlparse
from urllib.parse import parse_qs
from datetime import datetime

import numpy as np
import pandas as pd

# Data Challenge

Files are stored in an S3 bucket. The purpose here is to fully analyze the data and make some predictions.

This workbook was exported to a Python script and the resulting code was checked for PEP8 problems. Any problems were corrected.